In [4]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [5]:
# Initialize API key from .env and constants for models

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [6]:
# definition of our class for websites objects

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
   
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [7]:
test_david = Website("https://bitcoin.org/en/")

print(test_david.get_contents())

test_david.links

Webpage Title:
Bitcoin - Open source P2P money
Webpage Contents:
Bitcoin.org is a community funded project, donations are appreciated and used to improve the website.
Make a donation
Bitcoin.org needs your support!
×
Donate to Bitcoin.org
Use this QR code or address below
bc1qp6ejw8ptj9l9pkscmlf8fhhkrrjeawgpyjvtq8
$50.00
(... BTC)
$100.00
(... BTC)
$200.00
(... BTC)
Introduction
Individuals
Businesses
Developers
Getting started
How it works
White paper
Resources
Resources
Exchanges
Community
Documentation
Vocabulary
Events
Bitcoin Core
Innovation
Participate
Support Bitcoin
Buy Bitcoin
Sell Bitcoin
Running a full node
Development
FAQ
English
Bahasa Indonesia
Català
Dansk
Deutsch
English
Español
Français
Italiano
Magyar
Nederlands
Polski
Português Brasil
Română
Slovenščina
Srpski
Svenska
Türkçe
Ελληνικά
български
Русский
Українська
Հայերեն
العربية
فارسی
עברית
हिन्दी
한국어
ខ្មែរ
日本語
简体中文
繁體中文
Bahasa Indonesia
Català
Dansk
Deutsch
English
Español
Français
Italiano
Magyar
Nederlands
Polski
P

['bitcoin:bc1qp6ejw8ptj9l9pkscmlf8fhhkrrjeawgpyjvtq8',
 '/en/',
 '/en/bitcoin-for-individuals',
 '/en/bitcoin-for-businesses',
 'https://developer.bitcoin.org/',
 '/en/getting-started',
 '/en/how-it-works',
 '/en/bitcoin-paper',
 '/en/resources',
 '/en/exchanges',
 '/en/community',
 'https://developer.bitcoin.org/',
 '/en/vocabulary',
 '/en/events',
 '/en/bitcoin-core/',
 '/en/innovation',
 '/en/support-bitcoin',
 '/en/buy',
 '/en/sell',
 '/en/full-node',
 '/en/development',
 '/en/faq',
 '/id/',
 '/ca/',
 '/da/',
 '/de/',
 '/en/',
 '/es/',
 '/fr/',
 '/it/',
 '/hu/',
 '/nl/',
 '/pl/',
 '/pt_BR/',
 '/ro/',
 '/sl/',
 '/sr/',
 '/sv/',
 '/tr/',
 '/el/',
 '/bg/',
 '/ru/',
 '/uk/',
 '/hy/',
 '/ar/',
 '/fa/',
 '/he/',
 '/hi/',
 '/ko/',
 '/km/',
 '/ja/',
 '/zh_CN/',
 '/zh_TW/',
 '/en/getting-started',
 '/en/choose-your-wallet',
 '/en/buy',
 '/en/bitcoin-for-individuals',
 '/en/bitcoin-for-businesses',
 'https://developer.bitcoin.org/',
 '#maindesc-title',
 '/en/support-bitcoin',
 '/en/getting-s

In [8]:
# system prompts for LLM models
 
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [10]:
# func for a user prompt

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
print(get_links_user_prompt(test_david))

Here is the list of links on the website of https://bitcoin.org/en/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
bitcoin:bc1qp6ejw8ptj9l9pkscmlf8fhhkrrjeawgpyjvtq8
/en/
/en/bitcoin-for-individuals
/en/bitcoin-for-businesses
https://developer.bitcoin.org/
/en/getting-started
/en/how-it-works
/en/bitcoin-paper
/en/resources
/en/exchanges
/en/community
https://developer.bitcoin.org/
/en/vocabulary
/en/events
/en/bitcoin-core/
/en/innovation
/en/support-bitcoin
/en/buy
/en/sell
/en/full-node
/en/development
/en/faq
/id/
/ca/
/da/
/de/
/en/
/es/
/fr/
/it/
/hu/
/nl/
/pl/
/pt_BR/
/ro/
/sl/
/sr/
/sv/
/tr/
/el/
/bg/
/ru/
/uk/
/hy/
/ar/
/fa/
/he/
/hi/
/ko/
/km/
/ja/
/zh_CN/
/zh_TW/
/en/getting-started
/en/choose-your-wallet
/en/buy
/en/bitcoin-for-individuals
/en/bitcoin-for-businesses
https://developer.bitcoin.org

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# our website to check if models respond

huggingface = Website("https://huggingface.co")
huggingface.links

In [ ]:
get_links("https://huggingface.co")

In [ ]:
# func for making an easy brochure

def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
# show a result of brochure making func

print(get_all_details("https://huggingface.co"))

In [ ]:
# system prompt for model to make a brochure 

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.Please respond in Czech language."


In [ ]:
# func for making a complete brochure 

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("Trezor", "https://trezor.io/")

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("Trezor", "https://trezor.io/")